In [95]:
import winsound
import pygame
from bs4 import BeautifulSoup, NavigableString
import requests
from time import sleep
from random import randint
import math
from tqdm import tqdm
import pickle
import pandas as pd
from datetime import date
from datetime import datetime
import json

import pymysql                        # for getting data from a SQL database
from sqlalchemy import create_engine, text  # for establishing the connection and authentication
import zlib

from webscraper.scrap_modules import scraper_ds as mybib
from webscraper.scrap_modules import prep as prep

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
def get_job_information(soup):    
    class_soup = soup.find("ul", {"class":"description__job-criteria-list"}).find_all('span')
    list = []
    for x in class_soup:
        list.append(x.text.strip())
    
    return list

In [48]:
def get_all_job_information(id, response, scrap_date):

    soup = BeautifulSoup(response.content, "html.parser")
    job_dict = {}
    job_dict['id'] = id
    try:
        job_dict['title'] = soup.find('h2', class_='top-card-layout__title').text.strip()
    except:
        job_dict['title'] = None
        print("Error: in 'title'")
        
    try:    
        job_dict['company'] = soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
    except:
        job_dict['company'] = None
        print("Error: in 'company'")
        
    try:    
        job_dict['city'] = soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()
    except:
        job_dict['city'] = None
        print("Error: in 'city'")
    try:
        job_dict['posting_date'] = soup.find("span", {"class":"posted-time-ago__text"}).text.strip()
    except:
        job_dict['posting_date'] = None
        print("Error: in 'posting date'")
        
    try:    
        job_dict['job_description'] = soup.find("section",{"class":"show-more-less-html"}).text.strip()

    except:
        job_dict['job_description'] = None
        print("Error: in 'job description'")

    try:    
        x = get_job_information(soup)        
        job_dict['seniority_level'] = x[0]
        job_dict['employment_type'] = x[1]
        job_dict['job_function'] = x[2]
        job_dict['industries'] = x[3]
    except:
        print(f"Error: get job information in id {id}")
        job_dict['seniority_level'] = None
        job_dict['employment_type'] = None
        job_dict['job_function'] = None
        job_dict['industries'] = None
    
    try:    
        job_dict['scraping_date'] = scrap_date
    except:
        job_dict['scraping_date'] = None 
    
    try:
        name = job_dict['title']
        name = name.replace(" ", "%20")
        company = job_dict['company']
        company = company.replace(" ", "%20")
        job_dict['url'] = f"https://www.linkedin.com/jobs/search?keywords={name}%20{company}&location=Berlin%2C%20Berlin%2C%20Germany&geoId=&trk=public_jobs_jobs-search-bar_search-submit&position=1&pageNum=0"
    except:
        job_dict['url'] = None 
    
                        

    return job_dict

In [86]:
with open("webscraper/webscrap_data/soup_dict.pkl", "rb") as f:
    scraper_dict = pickle.load(f)

In [87]:
len(scraper_dict)

19776

In [96]:
df = prep.transform_scrap_data_to_df(scraper_dict)

NameError: name 'tqdm' is not defined

In [79]:
def transform_scrap_data_to_df(dict):
    columns = ['id','title', 'company', 'city', 'posting_date', 'job_description', 'seniority_level', 'employment_type', 'job_function', 'industries', 'scraping_date', 'url']
    df = pd.DataFrame(columns=columns)

    for key in tqdm(dict):
        id = key
        response = dict[key]['response']
        scrap_date = dict[key]["scrap_date"]
        new_dict = get_all_job_information(id, response, scrap_date)
        
        new_row_df = pd.DataFrame([new_dict])
        df = pd.concat([df, new_row_df], ignore_index=True)

    return(df)

In [54]:
scraper_df = transform_scrap_data_to_df(scraper_dict)

,id,title,company,city,posting_date,job_description,seniority_level,employment_type,job_function,industries,scraping_date,url
0,3685669741,Software Engineer Fullstack (m/w/d),Atruvia AG,"Aschheim, Bavaria, Germany",1 day ago,VollzeitWir sind der Digitalisierungspartner d...,Entry level,Full-time,Engineering and Information Technology,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
1,3719620327,Data Analyst - Business Intelligence,Almedia,"Berlin, Berlin, Germany",5 days ago,Almedia helps companies grow by promoting thei...,Mid-Senior level,Full-time,Analyst,"Technology, Information and Internet",2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
2,3720365801,(Junior) Data Engineer (m/f/d),Sandbox Interactive,"Berlin, Berlin, Germany",1 day ago,YOUR GAMEBuild and maintain the current data i...,Mid-Senior level,Full-time,Information Technology,Computer Games,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
3,3672508925,Financial Data Analyst (m/f/d),Ultramarin,"Berlin, Berlin, Germany",2 months ago,About The PositionWe are looking for a top-tie...,Mid-Senior level,Full-time,Information Technology,Financial Services,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
4,3695466781,Business Intelligence Analyst (f/m/d),Les Lunes,"Berlin, Berlin, Germany",4 weeks ago,Your missionAs a Business Intelligence Analyst...,Mid-Senior level,Full-time,"Research, Analyst, and Information Technology",Retail Apparel and Fashion,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
...,...,...,...,...,...,...,...,...,...,...,...,...
9114,3715652206,"Biologe, Informatiker - Forschungsdatenmanagem...",Bundesamt für Bauwesen und Raumordnung,"Cottbus, Brandenburg, Germany",1 week ago,"Das Bundesinstitut für Bau-, Stadt- und Raumfo...",Mid-Senior level,Full-time,Science,Biotechnology Research,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
9115,3706303616,Logistics Manager (f/m/x),Formo,"Berlin, Berlin, Germany",3 weeks ago,Formo is looking for a driven & experienced Lo...,Mid-Senior level,Full-time,Purchasing,Food and Beverage Services,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
9116,3708801807,Senior Manager / Director Data-Driven Manufact...,HJG Unternehmensberatungs GmbH,"Munich, Bavaria, Germany",3 weeks ago,"Unser Kunde, eine große, international tätige ...",Mid-Senior level,Full-time,Consulting,Business Consulting and Services,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
9117,3721309441,Webanalyst,schalk&friends,"Munich, Bavaria, Germany",1 week ago,WIR stellen einDu hast Freude an Zahlen und Er...,None,None,None,None,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...


In [55]:
with open("webscraper/webscrap_data/temp_df.pkl", "wb") as f:
    pickle.dump(scraper_df, f) 

In [68]:
columns = scraper_df.columns

for col in columns: 
    print(f"{scraper_df[col].isna().sum()} null values for {col}")
    

0 null values for id
12 null values for title
12 null values for company
12 null values for city
12 null values for posting_date
12 null values for job_description
121 null values for seniority_level
121 null values for employment_type
121 null values for job_function
121 null values for industries
0 null values for scraping_date
12 null values for url


### Second request empty rows


In [75]:
isnafilter = scraper_df[scraper_df["title"].isna()]
isnafilter

,id,title,company,city,posting_date,job_description,seniority_level,employment_type,job_function,industries,scraping_date,url
413,3723903886,None,None,None,None,None,None,None,None,None,2023-09-24,None
465,3723207688,None,None,None,None,None,None,None,None,None,2023-09-24,None
3350,3720181679,None,None,None,None,None,None,None,None,None,2023-09-24,None
8105,3701781374,None,None,None,None,None,None,None,None,None,2023-09-24,None
8329,3723937506,None,None,None,None,None,None,None,None,None,2023-09-24,None
8330,3725925289,None,None,None,None,None,None,None,None,None,2023-09-24,None
8838,3703776281,None,None,None,None,None,None,None,None,None,2023-09-24,None
8840,3703121938,None,None,None,None,None,None,None,None,None,2023-09-24,None
8869,3673669013,None,None,None,None,None,None,None,None,None,2023-09-24,None
8870,3707571067,None,None,None,None,None,None,None,None,None,2023-09-24,None


In [76]:
second_request_list = isnafilter["id"].tolist()

In [77]:
# second request for empty rows

second_request_dict = {}

for id in second_request_list:
    response = requests.get(f'https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{id}')
    wait_time = randint(1,500)
    sleep(wait_time/1000)

    dict2 = {}
    dict2["scrap_date"] = date.today()
    dict2["response"] = response
    second_request_dict[id] = dict2

In [78]:
second_request_dict

{'3723903886': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3723207688': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3720181679': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3701781374': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3723937506': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3725925289': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3703776281': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3703121938': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3673669013': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3707571067': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},
 '3711640042': {'scrap_date': datetime.date(2023, 9, 24),
  'response': <Response [200]>},

In [81]:
second_request_df = transform_scrap_data_to_df(second_request_dict)
second_request_df

100%|██████████| 12/12 [00:00<00:00, 60.17it/s]


,id,title,company,city,posting_date,job_description,seniority_level,employment_type,job_function,industries,scraping_date,url
0,3723903886,Kotlin Developer,Oliver Bernard,"Berlin, Berlin, Germany",3 days ago,Ever wanted to help fight crime by developing ...,Entry level,Full-time,Engineering and Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
1,3723207688,Software Engineer - Ruby (f/m/d),nebenan.de,"Berlin, Berlin, Germany",4 days ago,Your missionWe are looking for a Senior Backen...,Mid-Senior level,Full-time,Information Technology,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
2,3720181679,Portfolio Project Manager:in (w/m/d),Enpal,"Berlin, Berlin, Germany",1 week ago,UnternehmensbeschreibungMit Enpal verfolgen wi...,Entry level,Full-time,Analyst,Renewable Energy Semiconductor Manufacturing,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
3,3701781374,Teamleiter Cybersecurity und Data (All Genders),Expleo Group,"Stuttgart, Baden-Württemberg, Germany",2 days ago,Übersicht Für unseren Digital-Bereich suchen w...,Mid-Senior level,Full-time,Information Technology,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
4,3723937506,Web3 Lead Developer (m|w|d),join.com,Germany,3 days ago,Unser Kunde ist Knowledge Provider und Enabler...,Mid-Senior level,Full-time,Information Technology,Staffing and Recruiting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
5,3725925289,Test Engineer (all genders),Accenture DACH,"Kaiserslautern, Rhineland-Palatinate, Germany",1 day ago,"Topunternehmen brauchen eine IT, die jederzeit...",Entry level,Full-time,Information Technology and Engineering,IT Services and IT Consulting,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
6,3703776281,Senior Data Analyst (BI & Data Science) (m/w/d),DEVnet,"Grünwald, Bavaria, Germany",3 weeks ago,Shaping the Digital FutureWir sind ein aufgesc...,Mid-Senior level,Full-time,"Engineering, Information Technology, and Proje...","Computer Hardware Manufacturing, Computers and...",2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
7,3703121938,Wissenschaftliche/r Mitarbeiter/in (w/m/d),RWTH Aachen University,"Aachen, North Rhine-Westphalia, Germany",3 weeks ago,Ihr ProfilAbgeschlossenes Hochschulstudium (Ma...,Entry level,Full-time,"Research, Analyst, and Information Technology",Research Services,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
8,3673669013,NEW TECHNOLOGIES AND DIGITAL TRANSFORMATION CO...,Management Solutions,"Frankfurt, Hesse, Germany",5 days ago,Management Solutions is an international consu...,Entry level,Full-time,"Consulting, Management, and Information Techno...",Business Consulting and Services,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
9,3707571067,Implementation Specialist - Public transport (...,GIRO,"Gummersbach, North Rhine-Westphalia, Germany",2 weeks ago,Company DescriptionOur specialty is to optimiz...,Mid-Senior level,Full-time,Information Technology,Software Development,2023-09-24,https://www.linkedin.com/jobs/search?keywords=...
